<a target="_blank" href="https://colab.research.google.com/github/amanichopra/sap-genai-hub/blob/main/RAG.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Preparation

## Install Libraries

In [35]:
!pip install "generative-ai-hub-sdk[all]"
!pip install pdfplumber
!pip install hdbcli
!pip install langchain
!pip install boto3
!pip install ai-core-sdk
!pip install "numpy<2.0.0" --force-reinstall

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


Now, make sure to reset the runtime. In Google Colab, you can do this by clicking `Runtime` and `Restart Session`, as shown here:

<img src="assets/colab_restart_session.png" style="width:500px">

Now, you can continue by running the below cells. The packages have already been installed into the runtime before restarting.

## Authentication

Before requests to orchestration can be issued, we need to provide authentication details to the SDK. This can be done either via a configuration file or via the environment. Make sure to read the [Generative AI Hub SDK docs](https://help.sap.com/doc/generative-ai-hub-sdk/CLOUD/en-US/index.html) for more details. Below you will find an example for authenticating via environment variables using this very notebook. Ensure to store credentials in a file called `env_vars.env` file for the below command to work. If using Google Colab, you can place this file in the project folder by clicking the folder icon on the left and dropping the file in the workspace as shown:

<img src="./assets/upload_env.png" style="width:500px">

In [26]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path='env_vars.env')

True

# RAG

RAG (Retrieval-Augmented Generation) is a technique that enhances a language model by combining it with a retrieval system. It fetches relevant information from external sources (like documents or databases) and uses it to generate more accurate and informed responses.

Why RAG is useful:
- Improves factual accuracy and reduces hallucinations.
- Accesses external or private knowledge without retraining the model.
- Keeps models up-to-date without needing continuous fine-tuning.

LLMs have a fixed-size context window (e.g., 4k–128k tokens depending on the model), and this creates key limitations:
- They can’t handle very long documents or large corpora at once.
- They forget earlier parts of the input once the token limit is reached.
- They can’t retain large histories in ongoing conversations or workflows.
- Pushing large chunks of text into the model wastes compute and may reduce response quality.

RAG solves this by retrieving and inserting only the most relevant pieces, keeping the context window small and meaningful.

<img src="assets/rag_workflow.png" alt="Drawing" style="width: 800px;"/>


We will try to build an AI expert on meal planning and nutrition advice. The user should be able to ask the expert to generate a recipe based on the recipes he/she has made in the past. 

Suppose, we ask the expert to generate a recipe without context.

In [33]:
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.service import OrchestrationService

llm = LLM(
    name="gpt-4o",
    version="latest",
    parameters={"max_tokens": 1000, "temperature": 0.2},
)

template = Template(
    messages=[
        SystemMessage("You are a nutrition and meal planning expert. Your job is to suggest similar recipes based on what ingredients the user wants. " \
        "You will also be given a list of recipes he/she has made in the past as inspiration for new recipes the user likes."),
        UserMessage(
            "I want a recipe with at least the following ingredients: {{?ingr_string}}" \
            "Here are similar recipes I've made in the past: {{?sim_recipes}}",
        ),
    ]
)

config = OrchestrationConfig(
    template=template,
    llm=llm,
)

orchestration_service = OrchestrationService(
    deployment_id=os.environ['AICORE_ORCH_DEPLOYMENT_ID'],
    config=config,
)

result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="ingr_string",
            value="salmon, zuchinni, noodles",
        ),
        TemplateValue(
            name="sim_recipes",
            value='None',
        ),
    ]
)

print(result.orchestration_result.choices[0].message.content)

Great! Since you haven't made any similar recipes in the past, let's start with a simple and delicious dish that incorporates salmon, zucchini, and noodles. Here's a recipe for Salmon and Zucchini Noodle Stir-Fry:

### Salmon and Zucchini Noodle Stir-Fry

#### Ingredients:
- 2 salmon fillets (about 6 oz each)
- 2 medium zucchinis
- 8 oz noodles (such as soba, rice noodles, or spaghetti)
- 2 tablespoons olive oil
- 2 cloves garlic, minced
- 1 tablespoon soy sauce
- 1 tablespoon lemon juice
- 1 teaspoon honey or maple syrup
- Salt and pepper to taste
- Optional: sesame seeds and chopped green onions for garnish

#### Instructions:
1. **Prepare the Zucchini Noodles**: Use a spiralizer or a vegetable peeler to create zucchini noodles. Set aside.

2. **Cook the Noodles**: Cook the noodles according to the package instructions. Drain and set aside.

3. **Cook the Salmon**: Season the salmon fillets with salt and pepper. Heat 1 tablespoon of olive oil in a large skillet over medium-high heat.

The expert generates a plausable recipe, but I usually use shiritaki noodles when I make salmon, I don't add honey/maple syrup since I don't want sugar, and I really don't like Soy sause. Thus, it's important to pass context of the recipes I've made w/ Salmon, so that the expert understands my food preferences. First, we will export the data from MyFitnessPal where I have stored thousands of recipes that I've made.

In [2]:
import re
import pandas as pd
import pdfplumber

# Regex patterns
date_pattern = re.compile(r"([A-Z][a-z]+\s\d{1,2},\s\d{4})")  # e.g., May 9, 2022
meal_keywords = ["Breakfast", "Lunch", "Dinner", "Morning Snack", "Midday Snack", "Snack", "Afternoon Snack", "Evening Snack", "Brunch", "Late Night Snack"]

extraction = []
for path in ['rag_data/mfp_4-29-22_9-8-22.pdf', 'rag_data/mfp_4-26-24_4-26-25.pdf', 'rag_data/mfp_4-26-23_4-26-24.pdf']:

    # Open the PDF again for custom extraction
    records = []
    current_date = None
    current_meal = None

    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            lines = text.split("\n")
            
            for line in lines:
                line = line.strip()
                if not line:
                    continue

                # Check if line is a date
                date_match = date_pattern.match(line)
                if date_match:
                    current_date = date_match.group(1)
                    continue

                # Check if line is a meal section
                if any(meal.lower() in line.lower() for meal in meal_keywords):
                    current_meal = line.lower()
                    continue

                # Skip non-ingredient lines
                # Heuristic: ingredients lines have ingredient name, followed by comma, followed by quantitiy, and then followed by nutrition facts
                if current_date and current_meal and ',' in line and 'MyFitnessPal' not in line:
                    # Save the record
                    records.append({
                        "date": pd.to_datetime(current_date),
                        "meal_name": current_meal.replace("\n", " ").strip(),
                        "ingredient_name": line
                    })
    df = pd.DataFrame(records)
    extraction.append(df)
extraction = pd.concat(extraction)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

In [3]:
extraction.sort_values('date').head()

,date,meal_name,ingredient_name
0,2022-05-01,breakfast,"Black pepper, 1 tbsp, ground 17 4g 0g 1g 0mg 1..."
32,2022-05-01,morning snack,"Large Egg - One Large Egg, 3 each 210 0g 15g 1..."
31,2022-05-01,morning snack,"Spinach, 2 cup 14 2g 0g 2g 0mg 47mg 0g 1g"
30,2022-05-01,morning snack,"Egg Beater Whites - Kirkland, 92 g 50 0g 0g 10..."
29,2022-05-01,morning snack,"Generic - Red Onions, 60 g 20 5g 0g 1g -- 6mg ..."


Now let's create a string vector representation for each meal.

In [4]:
meals = extraction.groupby(['date', 'meal_name']).agg(list).reset_index()
newline = '\n'
meals['ingredient_name'] = meals['ingredient_name'].map(lambda x: f'Ingredients:\n{f"{newline}".join(sorted(x))}')
meals.shape

(285, 3)

In [5]:
print(meals.iloc[0, 2])

Ingredients:
Black pepper, 1 tbsp, ground 17 4g 0g 1g 0mg 1mg 0g 2g
Garlic powder, 1 tsp 10 2g 0g 1g 0mg 2mg 0g 0g
Green Chili Company - Hatch Green Chili, 22.5 g 9 2g 0g 0g 0mg 2mg 1g 0g
Happy Farms - Mozzarella Shredded Cheese, 0.5 cup 160 4g 12g 12g 30mg 360mg 2g 0g
Kale - Kale - Chopped, 85 g 42 9g 1g 3g 0mg 37mg 0g 2g
Kirkwood (Aldi) - Turkey, Ground, Raw, 85/15, 4 oz
Olive oil - Organic Olive Oil, 0.5 tbsp 60 0g 7g 0g 0mg 0mg 0g 0g
Spelt Bagel - Bagel, 1 bagel 280 46g 6g 8g 0mg 430mg 1g 6g


Now, to highlight why RAG is so useful, let's pass in all the recipes from the extraction as context to the model when asking it to generate a new recipe with our favorite ingredients.

In [34]:
result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="ingr_string",
            value="salmon, zuchinni, noodles",
        ),
        TemplateValue(
            name="sim_recipes",
            value='\n\n'.join(meals['ingredient_name'].tolist()*5),
        ),
    ]
)

print(result.orchestration_result.choices[0].message.content)

OrchestrationError: This model's maximum context length is 128000 tokens. However, your messages resulted in 296823 tokens. Please reduce the length of the messages.

Clearly, we see that the recipes we have extracted exceed the model's context window. Our options are to use a model with a larger context window such as `gemini-1.5-pro` which supports up to 2 million tokens, or use RAG. However, long-context still struggle with long contexts since there is information overload. Inference costs also tend to be much higher since there are more tokens being processed.

Let's proceed by loading the recipes into a HANA vector DB. First, we will establish a DB connection to HANA.

In [9]:
from hdbcli import dbapi

conn = dbapi.connect(
    address=os.environ.get("HANA_HOST"),
    port=os.environ.get("HANA_PORT"),
    user=os.environ.get("HANA_USER"),
    password=os.environ.get("HANA_PASSWORD"),
    autocommit=True,
    sslValidateCertificate=True,
)

Now, we will create a vector DB.

In [ ]:
def populate_db(db, text_chunks):
    """add documents to the vector db"""

    # add the loaded document chunks
    db.add_documents(text_chunks)

def clear_db(conn, table):
    cur = conn.cursor()
    cur.execute(f'DELETE FROM {table}')
    cur.close()
    return True

def drop_db(conn, table):
    cur = conn.cursor()
    cur.execute(f'DROP TABLE {table}')
    cur.close()
    return True

def query(conn, query):
    cur = conn.cursor()
    cur.execute(query)
    res = cur.fetchall()
    cur.close()
    return res

create_q = """CREATE COLUMN TABLE "INGREDIENT_EMBEDDINGS" (
    "VEC_TEXT" NCLOB MEMORY THRESHOLD 1000,
	"VEC_META" NCLOB MEMORY THRESHOLD 1000,
	"VEC_VECTOR" REAL_VECTOR
    )
    UNLOAD PRIORITY 5 AUTO MERGE;"""
print(drop_db(conn, 'INGREDIENT_EMBEDDINGS'))
query(conn, create_q)

Now, we'll load the vector DB with meals.

In [17]:
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings
from langchain_community.vectorstores.hanavector import HanaDB
from langchain.schema.document import Document

embedding_mod = OpenAIEmbeddings(proxy_model_name='text-embedding-3-small')

db = HanaDB(
    embedding=embedding_mod, connection=conn, table_name="INGREDIENT_EMBEDDINGS"
)
clear_db(conn, table='INGREDIENT_EMBEDDINGS')
print(query(conn, 'SELECT COUNT(*) from INGREDIENT_EMBEDDINGS'))
populate_db(db, [Document(page_content=row['ingredient_name'], metadata={'source': row['meal_name'], 'embedding_model': 'text-embedding-3-small', 'date': row['date'].strftime('%Y-%m-%d')}) for _, row in meals.iterrows()])
print(query(conn, 'SELECT COUNT(*) from INGREDIENT_EMBEDDINGS'))

[(0,)]
[(285,)]


Finally, let's test to make sure we can retrieve relevant meals given a query.

In [32]:
def query_db(db, query, topN=10):
    """retrieve the nearest neighbour based on cosine similarity"""
    docs = db.similarity_search(query, k=topN)
    return docs

query_db(db, 'salmon, zuchinni, noodles')

[Document(metadata={'source': 'morning snack', 'embedding_model': 'text-embedding-3-small', 'date': '2025-01-20'}, page_content='Ingredients:\nBlack pepper, 1 tbsp, ground 17 4g 0g 1g 0mg 1mg 0g 2g\nGarlic powder, 1 tsp 10 2g 0g 1g 0mg 2mg 0g 0g\nHomemade - Zuchinni Noodles, 200 grams 34 6g 1g 2g 0mg 16mg 5g 2g\nShiritaki - Noodles Lowcarb, 100 gram 20 4g 0g 1g 0mg 0mg 0g 6g\nSwad Chutney-Coriander - Chutney-Coriander, 2 tsp 9 1g 1g 0g -- 162mg 0g --\natlantic salmon - atlantic salmon, 187.5 g 315 0g 18g 38g 0mg 53mg 0g 0g\nhummus - Hummus, 4 tbsp 140 8g 10g 4g 0mg 260mg 2g 4g\norganic turmeric - turmeric, 1 tsp (9g) 8 1g 0g 0g 0mg 1mg 0g 1g'),
 Document(metadata={'source': 'morning snack', 'embedding_model': 'text-embedding-3-small', 'date': '2023-10-02'}, page_content="Ingredients:\nBlack pepper, 1 tbsp, ground 17 4g 0g 1g 0mg 1mg 0g 2g\nHomemade - Zuchinni Noodles, 200 grams 34 6g 1g 2g 0mg 16mg 5g 2g\nSwad Chutney-Coriander - Chutney-Coriander, 2 tsp 9 1g 1g 0g -- 162mg 0g --\nTrad

Finally, we can test the RAG workflow with orchestration.

In [30]:
def get_relevant_recipes(ingr_str):
    rel_docs = query_db(db, ingr_str)
    rel_docs = [d.page_content for d in rel_docs]
    return rel_docs

In [ ]:
ingr_str = 'salmon, zuchinni, noodles'
relevant_context = get_relevant_recipes(ingr_str)
result = orchestration_service.run(
    template_values=[
        TemplateValue(
            name="ingr_string",
            value=ingr_str,
        ),
        TemplateValue(
            name="sim_recipes",
            value='\n\n'.join(relevant_context),
        ),
    ]
)

print(result.orchestration_result.choices[0].message.content)

# Summary

In this exercise, you learned how RAG can be used to overcome the context window limits of LLMs. In addition, we learned how to leverage HANA's vector engine to set up a RAG pipeline. Continue to [Exercise 7](./agents.ipynb) where you will learn how to build agentic AI workflows using the Langgraph framework in Python!